<a href="https://colab.research.google.com/github/aTegart/abc-music-gen/blob/main/D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Getting set up:

You will be connecting to your personal google drive, so make sure you have '/content/drive/My Drive/CS 486 Music Generation/' present (without the ending space, so Celine you should edit your folder name).

Remember you need to authenticate every time you remount, and copy the auth code from the link.

Currently this code only reads from the montreal folder because google decided to be "helpful" and converted the folders of files from .txt to the google doc format at some point. I've tried changing my settings to prevent file type conversion and reuploaded the txt files I have on my computer, but no luck. Celine, since you found a file type converter solution before, maybe you have some ideas for how to make them stay txt files?



In [2]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Check position


In [25]:
!pwd
!cd './drive/My Drive/'
!pwd

/content
/content


## Preprocess the songs

In [26]:
from os import listdir
from os.path import isfile, join
import os

#list songs
mypath = '/content/drive/My Drive/CS 486 Music Generation/' #this was previously "myPath" when it needed to be "mypath" 🤯
#/content/drive/My Drive/CS 486 Music Generation 
#/content/drive/My Drive/CS 486 Music Generation
folders = [f for f in listdir(mypath) if not isfile(join(mypath, f))]
print(folders)
allSongs = []
for folder in folders:
  if folder == "montreal": #must fix file format for other folders :(
    for song in listdir(mypath + '/' + folder):
      f = open(os.path.join(mypath,folder,song), "r")
      text = f.read()
      allSongs.append(text)
print(len(allSongs))


['canad', 'montreal', 'tune arch canada']
219


In [28]:
print(allSongs[200])

X:139
T:Valse Saint Jean
C:Patrick Bouffard
M:3/4
K:D
A3 G F2 |A2 G2 F2 |GA B4 |B2 GA GF |G3 F E2 |G2 F2 E2 |FG A4 |A2 AB A^G |
A3 G F2 |A2 G2 F2 |GA B4 |c2 d2 B2 |A3 G F2 |A2 G2 E2 |1D6 |D6 :|2 D6 |D2 AB A^G |:
A2 AB A^G |A2 AB A^G |A^G AB AG |G2 GA GF |G2 GA GF |G2 GA GF |
GF GA Bc |B2 AB A^G |A2 AB A^G |A2 AB A^G |A^G AB AG |
G2 GA GF |G2 GA GF |G2 GA GF |GF GA Bc |1d2 AB A^G :|2d6 |]


In [29]:
#Combine songs into one long string
spacing = "\n\n"
combined_songs = spacing.join(allSongs)

#Find vocabulary for vectorization (unique characters in string)
uniq_chars = sorted(set(combined_songs))
print(len(uniq_chars))

char_num_map = {c:i for i, c in enumerate(uniq_chars)}
num_char_map = {i:c for i, c in enumerate(uniq_chars)}

#proof that the mapping holds both ways
print('{')
for char,_ in zip(char_num_map, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char_num_map[char]))
print('  ...\n}')

print('{')
for char,_ in zip(num_char_map, range(20)):
    print('  {:4s}: {:3s},'.format(repr(char), num_char_map[char]))
print('  ...\n}')

96
{
  '\n':   0,
  ' ' :   1,
  '"' :   2,
  '&' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  '+' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '/' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '4' :  16,
  '5' :  17,
  '6' :  18,
  '7' :  19,
  ...
}
{
  0   : 
  ,
  1   :    ,
  2   : "  ,
  3   : &  ,
  4   : '  ,
  5   : (  ,
  6   : )  ,
  7   : +  ,
  8   : ,  ,
  9   : -  ,
  10  : .  ,
  11  : /  ,
  12  : 0  ,
  13  : 1  ,
  14  : 2  ,
  15  : 3  ,
  16  : 4  ,
  17  : 5  ,
  18  : 6  ,
  19  : 7  ,
  ...
}


In [31]:
import numpy as np
#Vectorize combined_songs
# array of a num for each char in the combined_songs string
vsongs = np.array([char_num_map[c] for c in combined_songs])

print ('{} ---- characters mapped to int ----> {}'.format(repr(combined_songs[:10]), vsongs[:10]))
# check that vectorized_songs is a numpy array
assert isinstance(vsongs, np.ndarray), "returned result should be a numpy array"

'X:87\nT:Le ' ---- characters mapped to int ----> [50 22 20 19  0 46 22 38 62  1]


In [36]:
def batch(vsongs, seqlen, bsize):
  highest_start_index = vsongs.shape[0] - 1 - seqlen
  start_indices = np.random.choice(highest_start_index, bsize)

  input = [vsongs[i : i+seqlen] for i in start_indices]
  output = [vsongs[i+1 : i+1+seqlen] for i in start_indices]

  input = np.reshape(input, [bsize, seqlen])
  output = np.reshape(output, [bsize, seqlen])

  return input, output

testin, testout = batch(vsongs, 5, 2)
#observe how testout is shifted one num forward from testin
print(testin)
print(testout)

[[28 14 28 28  1]
 [28  1 27 14 33]]
[[14 28 28  1 28]
 [ 1 27 14 33 14]]


In [39]:
#more testing, remove later

x_batch, y_batch = batch(vsongs, seqlen=5, bsize=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
    print("Step {:3d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(num_char_map[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(num_char_map[target_idx])))

Step   0
  input: 69 ('l')
  expected output: 10 ('.')
Step   1
  input: 10 ('.')
  expected output: 60 ('c')
Step   2
  input: 60 ('c')
  expected output: 58 ('a')
Step   3
  input: 58 ('a')
  expected output: 0 ('\n')
Step   4
  input: 0 ('\n')
  expected output: 39 ('M')


## Setup Tensorflow and our ML env

In [27]:
%tensorflow_version 2.x
import tensorflow as tf 

# Check that we are using a GPU, if not switch runtimes
#   using Runtime > Change Runtime Type > GPU
assert len(tf.config.list_physical_devices('GPU')) > 0